In [0]:
DROP TABLE IF EXISTS bookstore.bronze.feeds;
DROP TABLE IF EXISTS bookstore.bronze.books;
DROP TABLE IF EXISTS bookstore.bronze.customers;
DROP TABLE IF EXISTS bookstore.bronze.orders;
DROP TABLE IF EXISTS bookstore.silver.books;
DROP TABLE IF EXISTS bookstore.silver.customers;
DROP TABLE IF EXISTS bookstore.silver.orders;
DROP DATABASE IF EXISTS bookstore.bronze CASCADE;
DROP DATABASE IF EXISTS bookstore.silver CASCADE;
DROP DATABASE IF EXISTS bookstore.gold CASCADE;
DROP CATALOG IF EXISTS bookstore CASCADE;

In [0]:
%python
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import AwsIamRole

workspace = WorkspaceClient()

In [0]:
%python
workspace.external_locations.delete('el_bookstorerawdata')
workspace.external_locations.delete('el_bookstoredatabricks')
workspace.storage_credentials.delete('sc_bookstorerawdata')
workspace.storage_credentials.delete('sc_bookstoredatabricks')

In [0]:
%python
workspace.storage_credentials.create(
    name='sc_bookstorerawdata'
    ,aws_iam_role=AwsIamRole(role_arn = 'arn:aws:iam::302263052839:role/databricks-s3-ingest-3fcda-db_s3_iam')
)
workspace.storage_credentials.create(
    name='sc_bookstoredatabricks'
    ,aws_iam_role=AwsIamRole(role_arn = 'arn:aws:iam::302263052839:role/databricks-s3-ingest-f00cb-db_s3_iam')
)

workspace.external_locations.create(
    name='el_bookstorerawdata'
    ,url='s3://bookstorerawdata/'
    ,credential_name='sc_bookstorerawdata'
)
workspace.external_locations.create(
    name='el_bookstoredatabricks'
    ,url='s3://bookstoredatabricks/'
    ,credential_name='sc_bookstoredatabricks'
)

In [0]:
CREATE CATALOG IF NOT EXISTS bookstore;
CREATE DATABASE IF NOT EXISTS bookstore.bronze;
CREATE DATABASE IF NOT EXISTS bookstore.silver;
CREATE DATABASE IF NOT EXISTS bookstore.gold;

In [0]:
CREATE TABLE IF NOT EXISTS bookstore.silver.books
(book_id STRING,title STRING,author STRING,price DOUBLE,start_date TIMESTAMP,end_date TIMESTAMP,isActive STRING,source_file STRING,insert_ts TIMESTAMP);

CREATE TABLE IF NOT EXISTS bookstore.silver.customers 
(customer_id STRING, email STRING, first_name STRING, last_name STRING, gender STRING, city STRING, country STRING, last_updated TIMESTAMP,source_file STRING,insert_ts TIMESTAMP);

CREATE TABLE IF NOT EXISTS bookstore.silver.orders
(order_id STRING, order_timestamp Timestamp, customer_id STRING, quantity BIGINT, total BIGINT, email STRING, first_name STRING, last_name STRING, gender STRING, street STRING, city STRING, country STRING, row_time TIMESTAMP, processed_timestamp TIMESTAMP, books ARRAY<STRUCT<book_id STRING, quantity BIGINT, subtotal BIGINT>>,source_file STRING,insert_ts TIMESTAMP);
